In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

from glob import glob
import numpy as np
from PIL import Image

veri seti: https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification


In [ ]:
train_dir_cats = "kedi_kopek/train/cats/*"
train_dir_dogs = "kedi_kopek/train/dogs/*"

train_dict = {"cats": [], "dogs": []}
for cat_file in glob(train_dir_cats):
    train_dict["cats"].append(cat_file)

for dog_file in glob(train_dir_dogs):
    train_dict["dogs"].append(dog_file)

test_dir_cats = "kedi_kopek/test/cats/*"
test_dir_dogs = "kedi_kopek/test/dogs/*"

test_dict = {"cats": [], "dogs": []}
for cat_file in glob(test_dir_cats):
    test_dict["cats"].append(cat_file)

for dog_file in glob(test_dir_dogs):
    test_dict["dogs"].append(dog_file)

In [ ]:
file_paths = []
labels = np.zeros(len(train_dict["cats"]) + len(train_dict["dogs"]))
labels[len(train_dict["cats"]):] = 1
file_paths.extend(train_dict["cats"])
file_paths.extend(train_dict["dogs"])
for i in [0, 200, 400, 500]:
    print(file_paths[i], labels[i])

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, files_dict):
        self.file_paths = []
        self.labels = np.zeros(
            len(train_dict["cats"]) + len(train_dict["dogs"]))
        self.labels[len(files_dict["cats"]):] = 1
        self.file_paths.extend(files_dict["cats"])
        self.file_paths.extend(files_dict["dogs"])
        self.transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.RandomCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.Resize(128),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        img = Image.open(self.file_paths[index])
        img = img.convert("RGB")
        img = self.transforms(img)
        label = labels[index]
        return img.numpy().astype("float32"), label.astype("long")

In [ ]:
train_dataset = CatsDogsDataset(train_dict)
BATCH_SIZE = 16
train_data_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=True)

In [ ]:
batch = next(iter(train_data_loader))[0]
print("Batch Shape:  ", batch.shape)

layer1 = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
x = layer1(batch)
print("After Layer 1:", x.shape)

layer2 = nn.Sequential(
    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2))
x = layer2(x)
print("After Layer 2:", x.shape)

In [ ]:
# NVIDIA GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CNN_1(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 2))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.fc(x)
        return x

In [ ]:
model_1 = CNN_1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model_1.parameters(), lr=0.001)

In [ ]:
num_batches = len(train_data_loader)

for epoch in range(10):
    avg_loss = 0

    for X, Y in train_data_loader:
        X = X.to(device)
        Y = Y.to(device)

        y_hat = model_1(X)
        loss = criterion(y_hat, Y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        avg_loss += loss.item()

    avg_loss = avg_loss / num_batches

    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

In [ ]:
test_dataset = CatsDogsDataset(test_dict)
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=True)

with torch.no_grad():
    model_1.to("cpu")
    total_predictions = 0
    num_correct_predictions = 0
    for X, Y in test_data_loader:
        predictions = model_1(X)
        predictions = predictions.argmax(dim=1)
        num_correct_predictions += (predictions == Y).sum()
        total_predictions += len(Y)

    print(num_correct_predictions.item(), "/", total_predictions)

In [ ]:
batch = next(iter(train_data_loader))[0]
print(batch.shape)

conv_layer_1 = nn.Sequential(
    nn.Conv2d(3, 64, 3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(2))
x = conv_layer_1(batch)
print("L1:", x.shape)

conv_layer_2 = nn.Sequential(
    nn.Conv2d(64, 512, 3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.MaxPool2d(2))
x = conv_layer_2(x)
print("L2:", x.shape)

conv_layer_3 = nn.Sequential(
    nn.Conv2d(512, 512, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.BatchNorm2d(512),
    nn.MaxPool2d(2))
x = conv_layer_3(x)
print("L3 - 1:", x.shape)

x = conv_layer_3(x)
print("L3 - 2:", x.shape)

x = conv_layer_3(x)
print("L3 - 3:", x.shape)

x = conv_layer_3(x)
print("L3 - 4:", x.shape)

In [ ]:
class CNN_2(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2))
        self.conv_layer_2 = nn.Sequential(
            nn.Conv2d(64, 512, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2))
        self.conv_layer_3 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=512*2*2, out_features=2))

    def forward(self, x):
        x = self.conv_layer_1(x)
        x = self.conv_layer_2(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.conv_layer_3(x)
        x = self.classifier(x)
        return x

In [ ]:
model_2 = CNN_2().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = Adam(model_2.parameters(), lr=0.01)

num_batches = len(train_data_loader)

for epoch in range(20):
    avg_loss = 0

    for X, Y in train_data_loader:
        X = X.to(device)
        Y = Y.to(device)

        y_hat = model_2(X)
        loss = criterion(y_hat, Y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        avg_loss += loss.item()

    avg_loss = avg_loss / num_batches

    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}")

In [ ]:
with torch.no_grad():
    model_2.to("cpu")
    model_2.eval()
    total_predictions = 0
    num_correct_predictions = 0
    for X, Y in test_data_loader:
        predictions = model_2(X)
        predictions = predictions.argmax(dim=1)
        num_correct_predictions += (predictions == Y).sum()
        total_predictions += len(Y)

    print(num_correct_predictions.item(), "/", total_predictions)